In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Initialise combined dataframe and get data source


In [ ]:
combined_df = pd.DataFrame()
excel_file = 'data/EDM 2022 Storm Overflow Annual Return - all water and sewerage companies.xlsx'
excel_data = pd.ExcelFile(excel_file)

## Heading cleanup

Headings are inconsistant in spelling and formatting. To resolve this issue, I am using the first sheet of headings. Also skipping the first row as it contains a workbook heading.

In [ ]:
first_sheet_name = excel_data.sheet_names[0]
first_sheet_df = pd.read_excel(excel_file, sheet_name=first_sheet_name, skiprows=1)
filters = first_sheet_df.columns.tolist()

Loops through each sheet in the excel_data

In [ ]:
for sheet_name in excel_data.sheet_names:
    sheet_df = pd.read_excel(excel_file, sheet_name=sheet_name, skiprows=1)
    filtered_df = sheet_df[filters]
    combined_df = combined_df._append(filtered_df, ignore_index=True)

print(combined_df)

In [ ]:
print(combined_df.columns)

Filter out uncessary columns

In [ ]:
selected_columns = ['Water Company Name', 
                    #'Site Name(EA Consents Database)', 
                    'Counted spills using 12-24h count method',
                    'Total Duration (hrs) all spills prior to processing through 12-24h count method'
                    ]
selected_df = combined_df[selected_columns]

print(selected_df)

Total Spill count by water company

In [ ]:
selected_df.loc[:, 'Total Duration (hrs) all spills prior to processing through 12-24h count method'] = pd.to_numeric(selected_df['Total Duration (hrs) all spills prior to processing through 12-24h count method'], errors='coerce')


selected_df['Total Duration (hrs) all spills prior to processing through 12-24h count method'] = selected_df['Total Duration (hrs) all spills prior to processing through 12-24h count method'].fillna(0)



print(selected_df)
# Group the data by 'Water Company Name' and calculate the total spills
grouped_df = selected_df.groupby('Water Company Name')['Total Duration (hrs) all spills prior to processing through 12-24h count method'].sum().round(2)
plt.figure(figsize=(20, 12), facecolor='white')
# Plot the bar chart
colours = plt.cm.Set3(np.arange(len(grouped_df)))
plt.bar(grouped_df.index, grouped_df.values, color=colours)
plt.xlabel('Water Company', fontsize=20, backgroundcolor='white')
plt.ylabel('Total Duration (hrs) of Sewage releases', fontsize =20, backgroundcolor='white')
plt.title('Total Duration (hrs) of Sewage releases by Water Company in 2022', fontsize =32, backgroundcolor='white')
for i, value in enumerate(grouped_df):
    plt.text(i, value, str(value), ha='center', va='bottom', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('Total Duration (hrs) of Sewage releases by Water Company.png in 2022.png', dpi=300, bbox_inches='tight')
plt.show()

